In [18]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import math

In [2]:

# # START NORMALIZING DATASET
DATA_PATH = os.path.join('Dataset_Wajah') 

actions = np.array(["Stop", "Maju", "Mundur", "Kanan", "Kiri" 
                    ])
no_sequences = 50
sequence_length = 5



In [3]:
def create_folder(actions, data_path, no_sequences):
    for action in actions: 
        for sequence in range(no_sequences):
            try: 
                os.makedirs(os.path.join(data_path, action, str(sequence)))
            except:
                pass


create_folder(actions, DATA_PATH, no_sequences)

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
mp_face_mesh = mp.solutions.face_mesh 

In [5]:
def mediapipe_detection(image, model):
    image.flags.writeable = False  # Mengubah gambar menjadi read-only untuk meningkatkan performa
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Konversi warna dari BGR (OpenCV) ke RGB
    results = model.process(image)  # Proses deteksi menggunakan model MediaPipe
    image.flags.writeable = True  # Membuat gambar kembali dapat ditulis
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Kembali ke format BGR untuk OpenCV
    return image, results

# Fungsi untuk menggambar face landmarks tanpa mata, mulut, dan hidung
def draw_face_landmarks(image, results):
    if results.face_landmarks:  # Memastikan bahwa ada hasil landmark wajah
        # Landmark mata, mulut, dan hidung yang akan dikecualikan
        excluded_landmarks = [
            1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,  # Kontur hidung, mulut, dan bibir
            33, 133, 362, 263,  # Mata
            168, 6  # Hidung bagian tengah
        ]

        # Koneksi original dari FACEMESH_CONTOURS
        custom_contours = list(mp_holistic.FACEMESH_CONTOURS)

        # Buang koneksi yang melibatkan landmark yang ingin dikecualikan
        custom_contours = [connection for connection in custom_contours if connection[0] not in excluded_landmarks and connection[1] not in excluded_landmarks]

        # Gambar custom landmarks tanpa mata, mulut, dan hidung
        mp_drawing.draw_landmarks(
            image, 
            results.face_landmarks, 
            custom_contours, 
            mp_drawing.DrawingSpec(thickness=1, circle_radius=1),  # Seting visual
            mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
        )

# Fungsi untuk mengekstrak keypoints tanpa mata, mulut, dan hidung
def extract_keypoints(results):
    # Landmark yang akan dikecualikan (definisi di awal, di luar blok if)
    excluded_landmarks = {
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,  # Kontur hidung, mulut, bibir
        33, 133, 362, 263,  # Mata
        168, 6  # Hidung bagian tengah
    }

    if results.face_landmarks:
        # Landmark dalam hasil face_landmarks
        landmarks = results.face_landmarks.landmark

        # Ekstraksi keypoints yang bukan bagian dari landmark yang dikecualikan
        keypoints = np.array([[landmark.x, landmark.y] for i, landmark in enumerate(landmarks) if i not in excluded_landmarks]).flatten()
    else:
        # Jika tidak ada face_landmarks, buat keypoints kosong dengan ukuran sesuai
        keypoints = np.zeros((468 - len(excluded_landmarks)) * 2)

    return keypoints



def wait_for_continue():
    cv2.putText(image, 'Nyantuy dikit teken G', (50, 100),  cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Tampilkan frame dengan teks
    cv2.imshow('OpenCV Feed', image)
    
    # Loop untuk menunggu input tanpa menghentikan video
    while True:
        if cv2.waitKey(1) & 0xFF == ord('g'):  # Menggunakan 1ms delay untuk terus menampilkan frame
            break


In [6]:
excluded_landmarks = {
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,  # Kontur hidung, mulut, bibir
    33, 133, 362, 263,  # Mata
    168, 6  # Hidung bagian tengah
}

# FACEMESH_CONTOURS adalah koneksi standar yang akan kita kurangi
import mediapipe as mp
mp_holistic = mp.solutions.holistic

# Mengambil koneksi FACEMESH_CONTOURS
facemesh_contours = list(mp_holistic.FACEMESH_CONTOURS)

# Buang koneksi yang melibatkan landmark yang dikecualikan
custom_contours = [connection for connection in facemesh_contours if connection[0] not in excluded_landmarks and connection[1] not in excluded_landmarks]

# Menghitung total jumlah landmark yang digunakan setelah pengecualian
used_landmarks = set()
for connection in custom_contours:
    used_landmarks.update(connection)

total_landmarks_used = len(used_landmarks)
print(f"Total landmark yang digunakan setelah pengurangan: {total_landmarks_used}")

Total landmark yang digunakan setelah pengurangan: 120


In [7]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):

                ret, frame = cap.read()

                image, results = mediapipe_detection(frame, holistic)
                
                # Gambar landmark pada frame sebelum ditampilkan
                draw_face_landmarks(image, results)
                # Tampilkan teks untuk memberitahu proses pengumpulan data
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,60), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(600)
                
                # Tampilkan teks saat sequence berikutnya dimulai
                elif frame_num == sequence_length - 1:
                    cv2.putText(image, 'NEXT SEQUENCE', (120,200), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(600)

                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,60), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)

                # Membuat latar belakang hitam
                black_bg = np.zeros((frame.shape[0], frame.shape[1], frame.shape[2]))

                # Gambar landmark pada black background (opsional)
                draw_face_landmarks(black_bg, results)

                # Simpan frame asli, frame dengan landmark, dan frame dengan latar hitam
                cv2.imwrite(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}-clear.jpg"), frame)
                cv2.imwrite(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.jpg"), image)
                cv2.imwrite(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}-black.jpg"), black_bg)
                
                # Ekstrak dan simpan keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Tampilkan frame yang sudah digambar landmarknya
                cv2.imshow('OpenCV Feed', image)

                # Memungkinkan pengguna untuk keluar dengan menekan 'q'
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            wait_for_continue()
                    
cap.release()
cv2.destroyAllWindows()

d:\TugasAkhir_FUji\.venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [8]:
coor_x = []
coor_y = []
coordinates = []

for action in actions:
     for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            for index, coor in enumerate(res):
                coordinates.append(coor)

                if(index % 2 != 0):
                    coor_x.append(coor)
                else:
                    coor_y.append(coor)

In [9]:
print(len(coordinates))

1117500


In [10]:
def normalization(midpoint_x, midpoint_y, shoulder_width, coordinates):
    print("Koordinat awal:")
    print("Max: " + str(max(coordinates)))
    print("Min: " + str(min(coordinates)))


    for index, coor in enumerate(coordinates):
        
        #position invariant
        if(index % 2 != 0):
            coordinates[index] -= midpoint_x
        else:
            coordinates[index] -= midpoint_y

        #scale invariant
        coordinates[index] /= shoulder_width

    print("\nKoordinat normalisasi:")
    print("Max: " + str(max(coordinates)))
    print("Min: " + str(min(coordinates)))
    return coordinates

def save_normalization(paths, actions, no_sequences, sequence_length, coor_norm):
    total_landmarks = 108
    temp_index = 0

    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                saved_coor_norm = coor_norm[temp_index:temp_index+total_landmarks]

                norm_npy_path = os.path.join(paths, action, str(sequence), str(frame_num) + "-norm")
                np.save(norm_npy_path, saved_coor_norm)

                temp_index += total_landmarks

In [19]:
midpoint_x = (coor_x[0] + coor_x[1])/2
midpoint_y = (coor_y[0] + coor_y[1])/2

# delta_x = midpoint_x - frame_width/2
# delta_y = midpoint_y - frame_height/2

shoulder_delta_x = coor_x[1] - coor_x[0]
shoulder_delta_y = coor_y[1] - coor_y[0]
shoulder_width = math.sqrt(pow(shoulder_delta_x, 2) + pow(shoulder_delta_y, 2))

In [21]:
normalized_coor = normalization(midpoint_x, midpoint_y, shoulder_width, coordinates)

save_normalization(DATA_PATH, actions, no_sequences, sequence_length, normalized_coor)

Koordinat awal:
Max: 0.8135124444961548
Min: 0.35463058948516846

Koordinat normalisasi:
Max: 5.650197559079372
Min: -6.968470133991347


In [22]:
check_coordinates = []

for action in actions:
     for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}-norm.npy".format(frame_num)))
            for index, coor in enumerate(res):
                check_coordinates.append(coor)

In [23]:
print(len(check_coordinates))
print("Max: " + str(max(check_coordinates)))
print("Min: " + str(min(check_coordinates)))

135000
Max: 4.238778705393544
Min: -6.679569425485824


In [ ]:
print(len(coor_x) * 2)
print(coor_x[0])
print(2*30*30*108)
print(12*2 + 21 * 2 * 2)

In [ ]:
len(actions)